In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -r yolov5/requirements.txt  # install dependencies
%cd yolov5

import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.5.0+cu101 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [ ]:
import os, shutil
shutil.copyfile('/content/drive/My Drive/newcoco.zip', 'coco.zip')
!unzip coco.zip
!mkdir /content/yolov5/data/labels/
!mkdir /content/yolov5/data/labels/coco_train/
!mkdir /content/yolov5/data/labels/coco_val/
!mkdir /content/yolov5/data/images/
!mv /content/yolov5/coco_train/ /content/yolov5/data/images/
!mv /content/yolov5/coco_val/ /content/yolov5/data/images/
!mv /content/yolov5/coco_val.json /content/yolov5/data/coco_val.json
!mv /content/yolov5/coco_train.json /content/yolov5/data/coco_train.json

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: coco_train/part2_001881.jpg  
  inflating: coco_train/part2_001884.jpg  
  inflating: coco_train/part2_001885.jpg  
  inflating: coco_train/part2_001886.jpg  
  inflating: coco_train/part2_001887.jpg  
  inflating: coco_train/part2_001888.jpg  
  inflating: coco_train/part2_001889.jpg  
  inflating: coco_train/part2_001890.jpg  
  inflating: coco_train/part2_001891.jpg  
  inflating: coco_train/part2_001892.jpg  
  inflating: coco_train/part2_001893.jpg  
  inflating: coco_train/part2_001894.jpg  
  inflating: coco_train/part2_001897.jpg  
  inflating: coco_train/part2_001898.jpg  
  inflating: coco_train/part2_001899.jpg  
  inflating: coco_train/part2_001900.jpg  
  inflating: coco_train/part2_001901.jpg  
  inflating: coco_train/part2_001902.jpg  
  inflating: coco_train/part2_001904.jpg  
  inflating: coco_train/part2_001910.jpg  
  inflating: coco_train/part2_001913.jpg  
  inflating: coco_train/part2_001914.jpg  
  inflating: coco_train/par

In [ ]:
from __future__ import print_function
import argparse
import glob
import os
import json
 
from PIL import Image

# json_files = sorted(glob.glob(os.path.join('/content/yolov5/data', '*.json')))  # 得到json文件路径下的所有json文件
set=['coco_train','coco_val']
for s in set:
    json_file='/content/yolov5/data/'+s+'.json'
    with open(json_file) as f:
        data = json.load(f)  # 将json文件转化为字典
        images = data['images']
        annotations = data['annotations']
        for img in images:
            id = img['id']
            width = float(img['width'])
            heigh = float(img['height'])
            converted_results = []
            for ann in annotations:
                if ann['image_id'] == id and ann['category_id'] <= 2:  # FLIR数据集中只有1-3
                    cat_id = int(ann['category_id'])
                    # letf top为左下角坐标 bbox_width bbox_height为目标框长宽
                    # 将bbox数值转化为float类型
                    left, top, bbox_width, bbox_height = map(float, ann['bbox'])  
                    # Yolo的id从0开始，FILIR从1开始
                    cat_id -= 1
                    # 求中心点坐标
                    x_center, y_center = (left + bbox_width / 2, top + bbox_height / 2)
                    # 归一化
                    x_rel=x_center/width
                    y_rel=y_center/heigh
                    w_rel=bbox_width/width
                    h_rel=bbox_height/heigh
                    
                    converted_results.append((cat_id, x_rel, y_rel, w_rel, h_rel))
                    if x_rel > 1 or y_rel>1 or w_rel>1 or h_rel>1:
                      print(ann['bbox'],(cat_id, x_rel, y_rel, w_rel, h_rel),width,heigh,img['file_name'])
            image_name = img['file_name']
            # 这里image_name是thermal_8_bit/FLIR_xxxxx.jpeg格式,我们文件名只需要FLIR_xxxxx部分
            image_name = os.path.basename(image_name)
            image_name,_=os.path.splitext(image_name)
            # print(image_name)  # 验证是名称否正确
            file = open('/content/yolov5/data/labels/' + s + '/' + str(image_name) + '.txt', 'w+')
            file.write('\n'.join('%d %.6f %.6f %.6f %.6f' % res for res in converted_results))
            file.close()

In [ ]:
!echo -e '# COCO 2017 dataset http://cocodataset.org \n train: /content/yolov5/data/images/coco_train  # 118k images \n val: /content/yolov5/data/images/coco_val  \n nc: 2 \n names: ['hat','person'] \n' > /content/yolov5/data/coco.yaml

In [ ]:
!sed -i "s/nc: 80/nc: 2/g" /content/yolov5/models/yolov5s.yaml
!cat /content/yolov5/models/yolov5s.yaml

# parameters
nc: 2  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# yolov5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 1-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 2-P2/4
   [-1, 3, Bottleneck, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 4-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 6-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]], # 8-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 6, BottleneckCSP, [1024]],  # 10
  ]

# yolov5 head
head:
  [[-1, 3, BottleneckCSP, [1024, False]],  # 11
   [-1, 1, nn.Conv2d, [na * (nc + 5), 1, 1]],  # 12 (P5/32-large)

   [-2, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 1, Conv, [512, 1, 1]],
   [-1, 3, BottleneckCSP, [5

In [ ]:
!python train.py --img 640 --batch 64 --epochs 300 --data /content/yolov5/data/coco.yaml --cfg /content/yolov5/models/yolov5s.yaml --weights '' --adam

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
{'lr0': 0.001, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
Your branch is behind 'origin/master' by 2 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)

Namespace(adam=True, batch_size=64, bucket='', cache_images=False, cfg='/content/yolov5/models/yolov5s.yaml', data='/content/yolov5/data/coco.yaml', device='', epochs=300, evolve=False, img_size=[640], multi_scale=False, name='', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

2020-06-17 17:05:55.801322: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Succes